In [1]:
import warnings
warnings.filterwarnings('ignore')
#import timeit
import json
import os
import numpy

from my_callback import MyCallback

Using TensorFlow backend.


In [2]:
import gym.spaces
import numpy
import pandas

LAG = 3

def calc_profit(action, df, index):
    if action == 0: # long
        p = 1
    elif action == 1: # short
        p = -1
    else: # stay
        p = 0
    return  p * df["c"][index]

def calc_observation(df, index, columns):
    return numpy.array(
        [
            [df[col][index-t] for col in columns] for t in range(LAG)
        ] 
    )

class Game(gym.core.Env):
    metadata = {'render.modes': ['human', 'rgb_array']}
    def __init__(self, df, columns):
        self.df = df.reset_index(drop=True)
        self.columns = columns
        self.action_space = gym.spaces.Discrete(3)
        self.observation_space = gym.spaces.Box(0, 999, shape=(LAG, len(columns)))
        self.time = LAG
        self.profit = 0
        
    def step(self, action):
        reward = calc_profit(action, self.df, self.time)
        self.time += 1
        self.profit += reward       
        done = self.time == (len(self.df) - 1)
        if done:
            print("profit___{}".format(self.profit))
        info = {}
        observation = calc_observation(self.df, self.time, self.columns)
        return observation, reward, done, info

    def reset(self):
        self.time = LAG
        self.profit = 0
        return calc_observation(self.df, self.time, self.columns)
    
    def render(self, mode):
        pass
    
    def close(self):
        pass
    
    def seed(self):
        pass


In [3]:
from keras.models import Sequential, Model
from keras.layers import (
    Dense,
    Activation,
    Flatten,
    Input,
    concatenate,
    Dropout,
    LSTM,
    Reshape
)

class Network(object):
    def __init__(self):
        self.model = None

    def sample_model(self, observation_space):
        model = Sequential()
        model.add(Reshape(observation_space.shape,
                      input_shape=(1,)+observation_space.shape))
        model.add(LSTM(LAG))
        model.add(Dense(32))
        model.add(Activation('relu'))
        #model.add(Dropout(0.6))
        model.add(Dense(16))
        model.add(Activation('relu'))
        #model.add(Dropout(0.6))
        model.add(Dense(n_action))
        model.add(Activation('linear'))
        self.model = model
        return model        
    
    def from_json(self, file_path):
        pass
    
    def to_json(self, output_path):
        pass


In [4]:
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory
from keras.optimizers import Adam

def agent(model, n_action):
    memory = SequentialMemory(limit=50000, window_length=1)
    policy = EpsGreedyQPolicy(eps=0.1)
    dqn_agent = DQNAgent(model=model, nb_actions=n_action,
                         memory=memory, nb_steps_warmup=100,
                         target_model_update=1e-2, policy=policy)
    dqn_agent.compile(Adam(lr=1e-3), metrics=['mae'])
    return dqn_agent

In [5]:
import pandas
import numpy

df = pandas.DataFrame({"a": numpy.random.rand(1000), "b": numpy.random.rand(1000)})
df["c"]= ((df["a"] - df["b"]).shift(2)).fillna(0)

In [6]:
df.head(10)

,a,b,c
0,0.064463,0.441743,0.000000
1,0.649705,0.019002,0.000000
2,0.372289,0.749011,-0.377280
3,0.190842,0.181702,0.630703
4,0.510036,0.970349,-0.376723
5,0.223740,0.027363,0.009140
6,0.532533,0.801741,-0.460313
7,0.635256,0.056010,0.196377
8,0.008362,0.570236,-0.269208
9,0.240967,0.896362,0.579246


In [7]:
columns = ["a", "b"]
env = Game(df, columns)
n_action = 3
network = Network()
model = network.sample_model(env.observation_space)
agent_v6 = agent(model, n_action)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [8]:
callback= MyCallback("tmp")
agent_v6.fit(env, nb_steps=100000, visualize=False,
                  verbose=2, callbacks=[callback])

Training for 100000 steps ...
Training for 100000 steps ...
profit___262.2135627716549
   996/100000: episode: 1, duration: 4.679s, episode steps: 996, steps per second: 213, episode reward: 262.214, mean reward: 0.263 [-0.923, 0.978], mean action: 0.583 [0.000, 2.000], loss: 0.028139, mean_absolute_error: 0.606626, mean_q: 1.204131
The reward is higher than the best one, saving checkpoint weights
   996/100000: episode: 1, duration: 4.726s, episode steps: 996, steps per second: 211, episode reward: 262.214, mean reward: 0.263 [-0.923, 0.978], mean action: 0.583 [0.000, 2.000], mean observation: 0.498 [0.000, 1.000], loss: 0.028139, mean_absolute_error: 0.606626, mean_q: 1.204131
profit___299.5220836642925
  1992/100000: episode: 2, duration: 3.673s, episode steps: 996, steps per second: 271, episode reward: 299.522, mean reward: 0.301 [-0.769, 0.971], mean action: 0.548 [0.000, 2.000], loss: 0.031391, mean_absolute_error: 2.282548, mean_q: 3.799711
The reward is higher than the best o

profit___279.8973996136567
 13944/100000: episode: 14, duration: 3.727s, episode steps: 996, steps per second: 267, episode reward: 279.897, mean reward: 0.281 [-0.941, 0.971], mean action: 0.602 [0.000, 2.000], loss: 0.230040, mean_absolute_error: 14.489065, mean_q: 21.947218
The reward is lower than the best one, checkpoint weights not updated
 13944/100000: episode: 14, duration: 3.728s, episode steps: 996, steps per second: 267, episode reward: 279.897, mean reward: 0.281 [-0.941, 0.971], mean action: 0.602 [0.000, 2.000], mean observation: 0.498 [0.000, 1.000], loss: 0.230040, mean_absolute_error: 14.489065, mean_q: 21.947218
profit___276.5946113535709
 14940/100000: episode: 15, duration: 3.700s, episode steps: 996, steps per second: 269, episode reward: 276.595, mean reward: 0.278 [-0.719, 0.978], mean action: 0.640 [0.000, 2.000], loss: 0.352726, mean_absolute_error: 14.894975, mean_q: 22.547369
The reward is lower than the best one, checkpoint weights not updated
 14940/100000

profit___284.21922660914703
 26892/100000: episode: 27, duration: 3.351s, episode steps: 996, steps per second: 297, episode reward: 284.219, mean reward: 0.285 [-0.676, 0.978], mean action: 0.657 [0.000, 2.000], loss: 0.293946, mean_absolute_error: 17.845539, mean_q: 27.035683
The reward is lower than the best one, checkpoint weights not updated
 26892/100000: episode: 27, duration: 3.352s, episode steps: 996, steps per second: 297, episode reward: 284.219, mean reward: 0.285 [-0.676, 0.978], mean action: 0.657 [0.000, 2.000], mean observation: 0.498 [0.000, 1.000], loss: 0.293946, mean_absolute_error: 17.845539, mean_q: 27.035683
profit___282.89321335671485
 27888/100000: episode: 28, duration: 3.317s, episode steps: 996, steps per second: 300, episode reward: 282.893, mean reward: 0.284 [-0.859, 0.978], mean action: 0.655 [0.000, 2.000], loss: 0.296379, mean_absolute_error: 17.972088, mean_q: 27.216761
The reward is lower than the best one, checkpoint weights not updated
 27888/1000

profit___273.25701703304463
 39840/100000: episode: 40, duration: 3.711s, episode steps: 996, steps per second: 268, episode reward: 273.257, mean reward: 0.274 [-0.862, 0.978], mean action: 0.652 [0.000, 2.000], loss: 0.447832, mean_absolute_error: 18.428686, mean_q: 27.823250
The reward is lower than the best one, checkpoint weights not updated
 39840/100000: episode: 40, duration: 3.712s, episode steps: 996, steps per second: 268, episode reward: 273.257, mean reward: 0.274 [-0.862, 0.978], mean action: 0.652 [0.000, 2.000], mean observation: 0.498 [0.000, 1.000], loss: 0.447832, mean_absolute_error: 18.428686, mean_q: 27.823250
profit___261.47756224909887
 40836/100000: episode: 41, duration: 3.620s, episode steps: 996, steps per second: 275, episode reward: 261.478, mean reward: 0.263 [-0.944, 0.978], mean action: 0.708 [0.000, 2.000], loss: 0.444542, mean_absolute_error: 18.470444, mean_q: 27.863611
The reward is lower than the best one, checkpoint weights not updated
 40836/1000

profit___262.0331442373136
 52788/100000: episode: 53, duration: 3.704s, episode steps: 996, steps per second: 269, episode reward: 262.033, mean reward: 0.263 [-0.934, 0.978], mean action: 0.733 [0.000, 2.000], loss: 0.367919, mean_absolute_error: 19.045084, mean_q: 28.793909
The reward is lower than the best one, checkpoint weights not updated
 52788/100000: episode: 53, duration: 3.705s, episode steps: 996, steps per second: 269, episode reward: 262.033, mean reward: 0.263 [-0.934, 0.978], mean action: 0.733 [0.000, 2.000], mean observation: 0.498 [0.000, 1.000], loss: 0.367919, mean_absolute_error: 19.045084, mean_q: 28.793909
profit___240.38341794506795
 53784/100000: episode: 54, duration: 3.703s, episode steps: 996, steps per second: 269, episode reward: 240.383, mean reward: 0.241 [-0.944, 0.978], mean action: 0.772 [0.000, 2.000], loss: 0.433145, mean_absolute_error: 19.072876, mean_q: 28.832588
The reward is lower than the best one, checkpoint weights not updated
 53784/10000

profit___261.4392559125526
 65736/100000: episode: 66, duration: 3.703s, episode steps: 996, steps per second: 269, episode reward: 261.439, mean reward: 0.262 [-0.934, 0.978], mean action: 0.682 [0.000, 2.000], loss: 0.391232, mean_absolute_error: 19.533293, mean_q: 29.527191
The reward is lower than the best one, checkpoint weights not updated
 65736/100000: episode: 66, duration: 3.704s, episode steps: 996, steps per second: 269, episode reward: 261.439, mean reward: 0.262 [-0.934, 0.978], mean action: 0.682 [0.000, 2.000], mean observation: 0.498 [0.000, 1.000], loss: 0.391232, mean_absolute_error: 19.533293, mean_q: 29.527191
profit___275.63107713929514
 66732/100000: episode: 67, duration: 3.746s, episode steps: 996, steps per second: 266, episode reward: 275.631, mean reward: 0.277 [-0.934, 0.978], mean action: 0.632 [0.000, 2.000], loss: 0.347654, mean_absolute_error: 19.485090, mean_q: 29.453693
The reward is lower than the best one, checkpoint weights not updated
 66732/10000

profit___273.4103869361664
 78684/100000: episode: 79, duration: 3.604s, episode steps: 996, steps per second: 276, episode reward: 273.410, mean reward: 0.275 [-0.727, 0.978], mean action: 0.702 [0.000, 2.000], loss: 0.384266, mean_absolute_error: 18.669577, mean_q: 28.262581
The reward is lower than the best one, checkpoint weights not updated
 78684/100000: episode: 79, duration: 3.605s, episode steps: 996, steps per second: 276, episode reward: 273.410, mean reward: 0.275 [-0.727, 0.978], mean action: 0.702 [0.000, 2.000], mean observation: 0.498 [0.000, 1.000], loss: 0.384266, mean_absolute_error: 18.669577, mean_q: 28.262581
profit___280.949437560194
 79680/100000: episode: 80, duration: 3.580s, episode steps: 996, steps per second: 278, episode reward: 280.949, mean reward: 0.282 [-0.826, 0.978], mean action: 0.662 [0.000, 2.000], loss: 0.305278, mean_absolute_error: 18.689735, mean_q: 28.323927
The reward is lower than the best one, checkpoint weights not updated
 79680/100000:

profit___270.24594978910517
 91632/100000: episode: 92, duration: 3.590s, episode steps: 996, steps per second: 277, episode reward: 270.246, mean reward: 0.271 [-0.862, 0.978], mean action: 0.706 [0.000, 2.000], loss: 0.251445, mean_absolute_error: 19.088770, mean_q: 28.910667
The reward is lower than the best one, checkpoint weights not updated
 91632/100000: episode: 92, duration: 3.591s, episode steps: 996, steps per second: 277, episode reward: 270.246, mean reward: 0.271 [-0.862, 0.978], mean action: 0.706 [0.000, 2.000], mean observation: 0.498 [0.000, 1.000], loss: 0.251445, mean_absolute_error: 19.088770, mean_q: 28.910667
profit___274.04368361237925
 92628/100000: episode: 93, duration: 3.591s, episode steps: 996, steps per second: 277, episode reward: 274.044, mean reward: 0.275 [-0.933, 0.978], mean action: 0.644 [0.000, 2.000], loss: 0.316830, mean_absolute_error: 19.223812, mean_q: 29.114166
The reward is lower than the best one, checkpoint weights not updated
 92628/1000

In [9]:
agent_v6.load_weights("tmp/best_weight.hdf5")

In [10]:
al = []
pl = []
for i in range(3, len(df)):
    obs = calc_observation(df, i, columns)
    action = agent_v6.forward(obs)
    profit = calc_profit(action, df, i)
    al.append(action)
    pl.append(profit)

In [11]:
df["pred"] = pandas.Series(al).shift(2)
df["profit"] = pandas.Series(pl).shift(3)

In [12]:
df

,a,b,c,pred,profit
0,0.064463,0.441743,0.000000,NaN,NaN
1,0.649705,0.019002,0.000000,NaN,NaN
2,0.372289,0.749011,-0.377280,0.0,NaN
3,0.190842,0.181702,0.630703,1.0,0.630703
4,0.510036,0.970349,-0.376723,1.0,0.376723
5,0.223740,0.027363,0.009140,1.0,-0.009140
6,0.532533,0.801741,-0.460313,0.0,0.460313
7,0.635256,0.056010,0.196377,1.0,0.196377
8,0.008362,0.570236,-0.269208,0.0,0.269208
9,0.240967,0.896362,0.579246,0.0,0.579246


In [13]:
df["c"].abs().sum(), df["profit"].sum()

(334.94402994906136, 296.80984002904404)